# 🎨 SDXL Base 1.0 - Google Colab クイックスタート

YouTube・Note用のサムネイル・アイコン生成に特化した商用利用可能なStable Diffusion XL画像生成システム

## 📋 このノートブックについて
- **モデル**: Stable Diffusion XL Base 1.0
- **ライセンス**: CreativeML Open RAIL++-M（商用利用可能）
- **対応環境**: Google Colab 無料版・Pro版
- **用途**: YouTube サムネイル、Note アイキャッチ、アイコン生成

## ⚡ クイックスタート手順
1. **GPU を有効化**: `ランタイム` → `ランタイムのタイプを変更` → `GPU`
2. **セル順次実行**: `Ctrl+F9` ですべてのセルを実行
3. **画像生成開始**: 最後のセクションでプロンプトを入力して生成

---

## 🔍 環境チェック
GPU が利用可能か確認します。

In [ ]:
# GPU・CUDA確認
!nvidia-smi

import torch
import sys

print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ GPU が利用できません。ランタイムタイプを GPU に変更してください。")

## 📦 自動セットアップ
SDXL Base 1.0 とすべての依存関係を自動インストールします。

In [ ]:
# リポジトリのクローン（必要な場合）
import os
import subprocess

if not os.path.exists('stable-diffusion-colab'):
    !git clone https://github.com/YOUR_USERNAME/stable-diffusion-colab.git
    %cd stable-diffusion-colab
else:
    print("✅ リポジトリは既に存在します")
    if not os.getcwd().endswith('stable-diffusion-colab'):
        %cd stable-diffusion-colab

In [ ]:
# 公式推奨による依存関係のインストール
print("🚀 SDXL Base 1.0 セットアップ開始...")

# 基本パッケージ（公式推奨順）
!pip install diffusers>=0.19.0 transformers safetensors accelerate

# SDXL専用ライブラリ
!pip install invisible_watermark

# PyTorch（CUDA対応）
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

# 画像処理・ユーティリティ
!pip install pillow opencv-python tqdm matplotlib

# オプション：高速化（失敗してもスキップ）
try:
    !pip install xformers
    print("✅ xformers インストール成功")
except:
    print("⚠️ xformers スキップ（オプション機能）")

print("✅ パッケージインストール完了")

## 🎨 SDXL パイプライン初期化
Stable Diffusion XL Base 1.0 を Google Colab 用に最適化して初期化します。

In [ ]:
import torch
from diffusers import DiffusionPipeline
import gc
import time
from IPython.display import display, Image as IPImage
from PIL import Image
import os

print("🔄 SDXL Base 1.0 モデル読み込み中...")
print("   (初回は数分かかる場合があります)")

# メモリクリア
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

# SDXL パイプライン初期化（公式推奨設定）
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,      # メモリ効率化
    use_safetensors=True,           # セキュリティ向上
    variant="fp16"                  # 軽量版モデル
)

print("✅ モデル読み込み完了")
print("⚡ Google Colab 最適化を適用中...")

# Google Colab 無料版用の必須最適化
pipe.enable_model_cpu_offload()   # CPU オフロード（公式推奨）
pipe.vae.enable_tiling()          # VAE タイリング

# オプション最適化（失敗時はスキップ）
try:
    pipe.enable_xformers_memory_efficient_attention()
    print("✅ xFormers メモリ最適化: 有効")
except:
    print("⚠️ xFormers メモリ最適化: 無効")

try:
    pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)
    print("✅ torch.compile 高速化: 有効")
except:
    print("⚠️ torch.compile 高速化: 無効")

print("🎉 SDXL パイプライン準備完了！")

# 出力ディレクトリ作成
os.makedirs("generated_images", exist_ok=True)
print("📁 generated_images/ フォルダを作成")

## 🧪 動作テスト
セットアップが正常に完了したかテスト画像を生成して確認します。

In [ ]:
print("🧪 テスト画像生成中...")

# テスト用プロンプト
test_prompt = "A beautiful landscape with mountains and a lake, digital art, high quality"

print(f"📝 プロンプト: {test_prompt}")
print("⏱️ 生成中... (1-2分お待ちください)")

start_time = time.time()

# テスト画像生成
test_images = pipe(
    prompt=test_prompt,
    height=1024,
    width=1024,
    num_inference_steps=20,      # 高速テスト
    guidance_scale=7.5,
    num_images_per_prompt=1
).images

generation_time = time.time() - start_time
test_image = test_images[0]

# 保存と表示
test_path = "generated_images/setup_test.png"
test_image.save(test_path)

print(f"✅ テスト完了！")
print(f"⏱️ 生成時間: {generation_time:.2f}秒")
print(f"📸 保存先: {test_path}")
print(f"📏 解像度: {test_image.size}")

# Colab で画像表示
display(test_image)

## 🎬 YouTube サムネイル生成
YouTube サムネイル用の画像を生成します（16:9 比率）。

In [ ]:
def generate_youtube_thumbnail(prompt, style="exciting", num_images=2):
    """YouTube サムネイル生成関数"""
    
    style_modifiers = {
        "exciting": "vibrant colors, high energy, dynamic composition, RGB lighting",
        "professional": "clean, modern, professional presentation, bright lighting",
        "gaming": "gaming setup, RGB lighting, action-packed, intense focus",
        "educational": "clear, educational, friendly atmosphere, engaging presentation"
    }
    
    full_prompt = f"""
    {prompt}, {style_modifiers.get(style, style_modifiers['exciting'])},
    YouTube thumbnail style, professional photography, ultra detailed,
    8K resolution, perfect composition
    """
    
    negative_prompt = """
    blurry, low quality, boring, plain, text overlay, watermark,
    dark, gloomy, unprofessional, amateur
    """
    
    print(f"🎬 YouTube サムネイル生成中...")
    print(f"📝 プロンプト: {prompt}")
    print(f"🎨 スタイル: {style}")
    
    start_time = time.time()
    
    images = pipe(
        prompt=full_prompt,
        negative_prompt=negative_prompt,
        height=576,                    # 16:9 比率
        width=1024,
        num_inference_steps=25,
        guidance_scale=8.0,
        num_images_per_prompt=num_images
    ).images
    
    generation_time = time.time() - start_time
    print(f"⏱️ 生成時間: {generation_time:.2f}秒")
    
    # 保存と表示
    for i, img in enumerate(images):
        filename = f"generated_images/youtube_thumbnail_{int(time.time())}_{i+1}.png"
        img.save(filename)
        print(f"💾 保存: {filename}")
        display(img)
    
    return images

# 使用例（自由にカスタマイズしてください）
youtube_prompt = "Excited gamer playing video game with colorful background"  # ← ここを変更
youtube_style = "gaming"  # exciting, professional, gaming, educational から選択

youtube_images = generate_youtube_thumbnail(youtube_prompt, youtube_style, 2)

## 📝 Note・ブログ記事用画像生成
Note やブログのアイキャッチ画像を生成します（1.91:1 比率）。

In [ ]:
def generate_blog_image(prompt, category="tech", num_images=2):
    """ブログ記事用画像生成関数"""
    
    category_modifiers = {
        "tech": "modern technology, clean workspace, professional setup",
        "lifestyle": "aesthetic, Instagram-worthy, natural lighting, cozy atmosphere",
        "business": "professional, corporate, clean design, business environment",
        "creative": "artistic, colorful, creative workspace, inspiring atmosphere"
    }
    
    full_prompt = f"""
    {prompt}, {category_modifiers.get(category, category_modifiers['tech'])},
    blog header image, professional photography, high quality,
    clean composition, detailed
    """
    
    negative_prompt = """
    cluttered, messy, unprofessional, low quality,
    text overlay, watermark, amateur, chaotic
    """
    
    print(f"📝 ブログ用画像生成中...")
    print(f"📝 プロンプト: {prompt}")
    print(f"🏷️ カテゴリ: {category}")
    
    start_time = time.time()
    
    images = pipe(
        prompt=full_prompt,
        negative_prompt=negative_prompt,
        height=536,                    # 1.91:1 比率
        width=1024,
        num_inference_steps=30,
        guidance_scale=7.5,
        num_images_per_prompt=num_images
    ).images
    
    generation_time = time.time() - start_time
    print(f"⏱️ 生成時間: {generation_time:.2f}秒")
    
    # 保存と表示
    for i, img in enumerate(images):
        filename = f"generated_images/blog_image_{int(time.time())}_{i+1}.png"
        img.save(filename)
        print(f"💾 保存: {filename}")
        display(img)
    
    return images

# 使用例（自由にカスタマイズしてください）
blog_prompt = "Modern laptop on clean desk with coffee cup and notebook"  # ← ここを変更
blog_category = "tech"  # tech, lifestyle, business, creative から選択

blog_images = generate_blog_image(blog_prompt, blog_category, 2)

## 🎨 アイコン・ロゴ生成
アイコンやロゴ用の画像を生成します（1:1 正方形）。

In [ ]:
def generate_icon(prompt, style="minimalist", num_images=3):
    """アイコン・ロゴ生成関数"""
    
    style_modifiers = {
        "minimalist": "simple, clean lines, minimal design, single color, geometric",
        "modern": "contemporary, sleek, professional, subtle gradients",
        "playful": "fun, colorful, friendly, approachable design",
        "elegant": "sophisticated, refined, luxury, premium quality"
    }
    
    full_prompt = f"""
    {prompt}, {style_modifiers.get(style, style_modifiers['minimalist'])},
    logo icon, scalable vector style, professional branding,
    clean background, symbol design, high quality
    """
    
    negative_prompt = """
    complex, detailed, photorealistic, cluttered, busy design,
    multiple colors, text, letters, shadows, 3D effects
    """
    
    print(f"🎨 アイコン生成中...")
    print(f"📝 プロンプト: {prompt}")
    print(f"🎭 スタイル: {style}")
    
    start_time = time.time()
    
    images = pipe(
        prompt=full_prompt,
        negative_prompt=negative_prompt,
        height=1024,                   # 正方形
        width=1024,
        num_inference_steps=35,
        guidance_scale=9.0,
        num_images_per_prompt=num_images
    ).images
    
    generation_time = time.time() - start_time
    print(f"⏱️ 生成時間: {generation_time:.2f}秒")
    
    # 保存と表示
    for i, img in enumerate(images):
        filename = f"generated_images/icon_{int(time.time())}_{i+1}.png"
        img.save(filename)
        print(f"💾 保存: {filename}")
        display(img)
    
    return images

# 使用例（自由にカスタマイズしてください）
icon_prompt = "Technology innovation symbol with geometric shapes"  # ← ここを変更
icon_style = "minimalist"  # minimalist, modern, playful, elegant から選択

icon_images = generate_icon(icon_prompt, icon_style, 3)

## 🎯 カスタム画像生成
自由なプロンプトで画像を生成できます。設定を細かく調整可能です。

In [ ]:
# カスタム設定（自由に変更してください）
custom_prompt = "A futuristic city at sunset with flying cars"  # ← ここにプロンプトを入力

custom_negative = "blurry, low quality, distorted"  # ← ネガティブプロンプト

# 画像設定
custom_height = 1024        # 高さ
custom_width = 1024         # 幅
custom_steps = 30           # 推論ステップ数（多いほど高品質、時間もかかる）
custom_guidance = 7.5       # ガイダンススケール（高いほどプロンプトに忠実）
custom_num_images = 2       # 生成画像数

print(f"🎯 カスタム画像生成中...")
print(f"📝 プロンプト: {custom_prompt}")
print(f"📏 サイズ: {custom_width}×{custom_height}")
print(f"⚙️ ステップ数: {custom_steps}")

start_time = time.time()

custom_images = pipe(
    prompt=custom_prompt,
    negative_prompt=custom_negative,
    height=custom_height,
    width=custom_width,
    num_inference_steps=custom_steps,
    guidance_scale=custom_guidance,
    num_images_per_prompt=custom_num_images
).images

generation_time = time.time() - start_time
print(f"⏱️ 生成時間: {generation_time:.2f}秒")

# 保存と表示
for i, img in enumerate(custom_images):
    filename = f"generated_images/custom_{int(time.time())}_{i+1}.png"
    img.save(filename)
    print(f"💾 保存: {filename}")
    display(img)

## 🔄 バッチ処理
複数のプロンプトを一度に処理できます。

In [ ]:
# バッチ処理用プロンプトリスト（自由に追加・変更してください）
batch_prompts = [
    {
        "name": "tech_workspace",
        "prompt": "Modern programmer workspace with multiple monitors and RGB lighting",
        "type": "youtube"  # youtube, blog, icon
    },
    {
        "name": "nature_landscape", 
        "prompt": "Beautiful mountain landscape with lake reflection at golden hour",
        "type": "blog"
    },
    {
        "name": "abstract_logo",
        "prompt": "Abstract geometric logo with clean lines and modern design",
        "type": "icon"
    }
]

def batch_generate(prompts_list):
    """バッチ処理関数"""
    
    type_configs = {
        "youtube": {"height": 576, "width": 1024, "steps": 25, "guidance": 8.0},
        "blog": {"height": 536, "width": 1024, "steps": 30, "guidance": 7.5},
        "icon": {"height": 1024, "width": 1024, "steps": 35, "guidance": 9.0}
    }
    
    results = []
    
    for i, item in enumerate(prompts_list):
        print(f"\n🔄 {i+1}/{len(prompts_list)}: {item['name']}")
        print(f"📝 {item['prompt']}")
        
        config = type_configs.get(item['type'], type_configs['blog'])
        
        try:
            start_time = time.time()
            
            images = pipe(
                prompt=item['prompt'],
                negative_prompt="blurry, low quality, unprofessional",
                height=config['height'],
                width=config['width'],
                num_inference_steps=config['steps'],
                guidance_scale=config['guidance'],
                num_images_per_prompt=1
            ).images
            
            generation_time = time.time() - start_time
            
            # 保存
            filename = f"generated_images/batch_{item['name']}_{int(time.time())}.png"
            images[0].save(filename)
            
            print(f"✅ 完了 ({generation_time:.2f}秒) - {filename}")
            display(images[0])
            
            results.append({
                'name': item['name'],
                'success': True,
                'time': generation_time,
                'file': filename
            })
            
            # メモリクリア
            torch.cuda.empty_cache()
            
        except Exception as e:
            print(f"❌ エラー: {e}")
            results.append({
                'name': item['name'],
                'success': False,
                'error': str(e)
            })
    
    return results

# バッチ処理実行
print("🚀 バッチ処理開始...")
batch_results = batch_generate(batch_prompts)

# 結果サマリー
successful = sum(1 for r in batch_results if r['success'])
total_time = sum(r.get('time', 0) for r in batch_results)

print(f"\n📊 バッチ処理完了:")
print(f"   ✅ 成功: {successful}/{len(batch_results)}")
print(f"   ⏱️ 総時間: {total_time:.2f}秒")
print(f"   📁 保存先: generated_images/")

## 🧹 メモリ管理
メモリ使用量の確認とクリーンアップを行います。

In [ ]:
def check_memory_usage():
    """メモリ使用量チェック"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1e9
        reserved = torch.cuda.memory_reserved() / 1e9
        total = torch.cuda.get_device_properties(0).total_memory / 1e9
        
        print(f"📊 GPU メモリ使用状況:")
        print(f"   使用中: {allocated:.2f}GB")
        print(f"   予約済み: {reserved:.2f}GB")
        print(f"   総容量: {total:.2f}GB")
        print(f"   使用率: {(allocated/total)*100:.1f}%")
        
        if allocated/total > 0.8:
            print("⚠️ メモリ使用量が高いです。クリーンアップを実行してください。")
    else:
        print("GPU が利用できません")

def cleanup_memory():
    """メモリクリーンアップ"""
    print("🧹 メモリクリーンアップ中...")
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    gc.collect()
    
    print("✅ クリーンアップ完了")

# メモリ状況確認
check_memory_usage()

# 必要に応じてクリーンアップ実行
cleanup_memory()

# クリーンアップ後の状況
print("\nクリーンアップ後:")
check_memory_usage()

## 💡 使用のコツとトラブルシューティング

### 🎯 効果的なプロンプトのコツ
- **具体的に**: 「美しい風景」→「夕日に照らされた山と湖の風景」
- **スタイル指定**: "digital art", "professional photography", "minimalist design"
- **品質キーワード**: "high quality", "ultra detailed", "8K resolution"

### ⚡ パフォーマンス最適化
- **解像度調整**: 大きすぎる解像度は避ける
- **ステップ数**: 20-30で十分な品質
- **バッチサイズ**: 無料版では1-2枚ずつ生成

### 🛠️ よくある問題の解決法

#### CUDA Out of Memory エラー
```python
# 解像度を下げる
height, width = 768, 768

# メモリクリーンアップ
torch.cuda.empty_cache()
gc.collect()

# より積極的なCPUオフロード
pipe.enable_sequential_cpu_offload()
```

#### 生成速度が遅い場合
```python
# ステップ数を減らす
num_inference_steps = 15

# DPMSolver使用
from diffusers import DPMSolverMultistepScheduler
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
```

### 📱 用途別推奨設定

| 用途 | 解像度 | ステップ数 | ガイダンス |
|------|--------|------------|------------|
| YouTube | 1024×576 | 25 | 8.0 |
| ブログ | 1024×536 | 30 | 7.5 |
| アイコン | 1024×1024 | 35 | 9.0 |
| テスト | 768×768 | 15 | 7.0 |

---

## 🎉 セットアップ完了！

これで SDXL Base 1.0 を使った商用利用可能な画像生成システムが準備できました。

### 📋 次のステップ
1. 上記のセルを使って様々な画像を生成してみてください
2. プロンプトを工夫して理想的な画像を作成
3. generated_images/ フォルダから画像をダウンロード

### 💼 商用利用について
- **ライセンス**: CreativeML Open RAIL++-M
- **用途**: アート・教育・研究目的での商用利用が可能
- **加工**: 生成された画像の自由な編集・配布が可能

---

**🚀 素晴らしい画像生成をお楽しみください！**